# Reproduce Main Results

This notebook reproduces the key findings from the temporal awareness research.

**Time to run**: ~5 minutes (no GPU required)

**Claims verified**:
1. Temporal scope is linearly encoded (92.5% probe accuracy)
2. Steering correlates with probe predictions (r=0.935)
3. Late layers encode semantic features (robust to keyword removal)

In [ ]:
import json
import pickle
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Paths
ROOT = Path("..").resolve()
DATA_DIR = ROOT / "data" / "raw"
CHECKPOINTS_DIR = ROOT / "results" / "checkpoints"
FIGURES_DIR = ROOT / "results" / "figures"
FIGURES_DIR.mkdir(exist_ok=True)

print(f"Root: {ROOT}")
print(f"Checkpoints: {list(CHECKPOINTS_DIR.glob('*.pkl'))[:3]}...")

## 1. Load Datasets

In [ ]:
# Load explicit (train) and implicit (test) datasets
with open(DATA_DIR / "temporal_scope_caa.json") as f:
    explicit_data = json.load(f)
    
with open(DATA_DIR / "temporal_scope_implicit.json") as f:
    implicit_data = json.load(f)

print(f"Explicit pairs: {len(explicit_data.get('pairs', []))}")
print(f"Implicit pairs: {len(implicit_data.get('pairs', []))}")

# Show sample
print("\nSample explicit pair:")
sample = explicit_data['pairs'][0]
print(f"  Question: {sample['question'][:50]}...")
print(f"  Immediate: {sample['immediate'][:50]}...")
print(f"  Long-term: {sample['long_term'][:50]}...")

## 2. Load Pre-trained Probes

In [ ]:
# Load probes for all layers
probes = {}
for layer in range(12):
    probe_path = CHECKPOINTS_DIR / f"temporal_caa_layer_{layer}_probe.pkl"
    if probe_path.exists():
        with open(probe_path, "rb") as f:
            probes[layer] = pickle.load(f)

print(f"Loaded probes for layers: {list(probes.keys())}")

## 3. Load Steering Vectors

In [ ]:
# Load learned steering vectors
steering_path = CHECKPOINTS_DIR / "temporal_directions_learned.json"
with open(steering_path) as f:
    steering_data = json.load(f)

print(f"Steering vector metadata:")
print(f"  Dimension: {steering_data.get('dimension', 'N/A')}")
print(f"  Model: {steering_data.get('model', 'N/A')}")
print(f"  Layers: {len(steering_data.get('vectors', {}))}")

## 4. Claim 1: Probe Accuracy by Layer

**Expected**: Peak accuracy ~92.5% at Layer 8 (training), ~84% at Layer 6 (test)

In [ ]:
# Note: This requires activations to be extracted first
# For quick verification, we report the stored results

# These are the reported results - need verification with fresh extraction
reported_train_acc = [68.0, 74.8, 78.5, 83.8, 87.5, 89.0, 91.0, 91.5, 92.5, 90.0, 87.5, 85.3]
reported_test_acc = [72.0, 65.0, 73.0, 80.0, 82.0, 83.0, 84.0, 81.0, 81.0, 76.0, 74.0, 77.0]

fig, ax = plt.subplots(figsize=(10, 6))
layers = list(range(12))

ax.plot(layers, reported_train_acc, 'b-o', label='Training (reported)', linewidth=2)
ax.plot(layers, reported_test_acc, 'r-s', label='Test (reported)', linewidth=2)
ax.axhline(y=50, color='gray', linestyle='--', label='Chance')

ax.set_xlabel('Layer', fontsize=12)
ax.set_ylabel('Accuracy (%)', fontsize=12)
ax.set_title('Temporal Scope Detection by Layer (⚠️ Needs Verification)', fontsize=14)
ax.legend()
ax.set_ylim(40, 100)
ax.grid(True, alpha=0.3)

# Mark peaks
ax.annotate(f'Peak: {max(reported_train_acc)}%', 
            xy=(reported_train_acc.index(max(reported_train_acc)), max(reported_train_acc)),
            xytext=(10, -20), textcoords='offset points', fontsize=10)

plt.tight_layout()
plt.savefig(FIGURES_DIR / "layer_accuracy.png", dpi=150)
plt.show()

print(f"\n✓ Figure saved to {FIGURES_DIR / 'layer_accuracy.png'}")
print(f"\n⚠️ VERIFICATION NEEDED: Run full extraction to confirm these values")

## 5. Claim 2: Steering Correlation

**Expected**: r=0.935 correlation between steering strength and probe predictions at Layer 11

In [ ]:
# Reported correlations by layer
reported_correlations = {
    0: -0.067, 1: 0.118, 2: 0.011, 3: 0.040,
    4: 0.492, 5: 0.580, 6: 0.723, 7: 0.812,
    8: 0.838, 9: 0.914, 10: 0.930, 11: 0.935
}

fig, ax = plt.subplots(figsize=(10, 6))

layers = list(reported_correlations.keys())
correlations = list(reported_correlations.values())

colors = ['red' if c < 0.5 else 'orange' if c < 0.7 else 'green' for c in correlations]
ax.bar(layers, correlations, color=colors, alpha=0.7, edgecolor='black')

ax.axhline(y=0.7, color='green', linestyle='--', alpha=0.5, label='Strong (r>0.7)')
ax.axhline(y=0, color='gray', linestyle='-', alpha=0.3)

ax.set_xlabel('Layer', fontsize=12)
ax.set_ylabel('Correlation (r)', fontsize=12)
ax.set_title('Steering-Probe Correlation by Layer (⚠️ Needs Verification)', fontsize=14)
ax.set_ylim(-0.2, 1.0)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(FIGURES_DIR / "steering_correlation.png", dpi=150)
plt.show()

print(f"\n✓ Figure saved to {FIGURES_DIR / 'steering_correlation.png'}")
print(f"Peak correlation: Layer 11, r={reported_correlations[11]}")
print(f"\n⚠️ VERIFICATION NEEDED: Run steering experiment to confirm")

## 6. Claim 3: Keyword Ablation (Semantic vs Lexical)

**Expected**: Late layers (10-11) achieve 100% accuracy even without temporal keywords

In [ ]:
# Reported ablation results
reported_original = [72, 65, 73, 80, 82, 83, 84, 81, 81, 76, 74, 77]
reported_ablated = [52, 55, 64, 68, 70, 72, 78, 85, 91, 99, 100, 100]

fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(12)
width = 0.35

ax.bar(x - width/2, reported_original, width, label='With keywords', color='steelblue', alpha=0.8)
ax.bar(x + width/2, reported_ablated, width, label='Keywords removed', color='coral', alpha=0.8)

ax.axhline(y=50, color='gray', linestyle='--', label='Chance', alpha=0.5)

ax.set_xlabel('Layer', fontsize=12)
ax.set_ylabel('Accuracy (%)', fontsize=12)
ax.set_title('Keyword Ablation: Lexical vs Semantic Encoding (⚠️ Needs Verification)', fontsize=14)
ax.set_xticks(x)
ax.legend()
ax.set_ylim(40, 105)
ax.grid(True, alpha=0.3, axis='y')

# Annotate key finding
ax.annotate('Semantic encoding\n(robust to keyword removal)', 
            xy=(10.5, 100), xytext=(7, 60),
            arrowprops=dict(arrowstyle='->', color='black'),
            fontsize=10, ha='center')

plt.tight_layout()
plt.savefig(FIGURES_DIR / "ablation.png", dpi=150)
plt.show()

print(f"\n✓ Figure saved to {FIGURES_DIR / 'ablation.png'}")
print(f"\nKey finding: Layers 10-11 achieve 100% accuracy WITHOUT keywords")
print(f"This suggests semantic (not lexical) encoding in late layers")
print(f"\n⚠️ VERIFICATION NEEDED: This is the most surprising claim - needs careful validation")

## 7. Summary

In [ ]:
print("="*60)
print("RESULTS SUMMARY")
print("="*60)
print()
print("Claim 1: Probe Accuracy")
print(f"  - Reported train peak: {max(reported_train_acc)}% (Layer {reported_train_acc.index(max(reported_train_acc))})")
print(f"  - Reported test peak: {max(reported_test_acc)}% (Layer {reported_test_acc.index(max(reported_test_acc))})")
print(f"  - Status: ⚠️ NEEDS VERIFICATION")
print()
print("Claim 2: Steering Correlation")
print(f"  - Reported peak: r={max(reported_correlations.values())} (Layer 11)")
print(f"  - Status: ⚠️ NEEDS VERIFICATION")
print()
print("Claim 3: Semantic Encoding")
print(f"  - Reported: 100% accuracy on ablated data (Layers 10-11)")
print(f"  - Status: ⚠️ NEEDS VERIFICATION (most surprising claim)")
print()
print("="*60)
print("To fully verify, run: python scripts/verify_all_claims.py")
print("="*60)

## Next Steps

1. **Verify claims**: Run `make verify` to extract fresh activations and validate
2. **Audit datasets**: Check for keyword leakage in implicit test set
3. **Cross-model**: Test on GPT-2 medium/large, Pythia
4. **Extend**: Implement intertemporal preference framework from research plan